In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation one our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    gamma = discount_factor
    pi = policy
    while True:        
        delta = 0        
        for s in range(env.nS):            
            Vea = 0 # used to compute Expectation over all actions  
            
            for a in range(env.nA): # For each possible action in - s
                Vq = 0 # q-value of current [s,a]                
                for P,St1,Ra,_ in env.P[s][a]: # every possible s' wind might blow you after taking a from s
                    Vq += P*V[St1] ## In-Place-DP
#                     Vq += P*Vold[St1] ## Normal DP (two copies of value function)
                Vq = Ra + gamma*Vq
                Vea += pi[s,a]*Vq 
        
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(Vea - V[s]))            
            V[s] = Vea # update new value for state - s            
       
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
        
    return np.array(V)


In [4]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    gamma = discount_factor
    
    while True:
        # Implement this!
        # Evaluate current PI
        Vpi = policy_eval_fn(policy, env, discount_factor)
        
        q = np.zeros([env.nS, env.nA])
        for s in range(env.nS):
            # Compute state-action function q for all a's of current s
            for a in range(env.nA):                
                for P,St1,Ra,_ in env.P[s][a]:
                    q[s][a] += gamma*P*Vpi[St1] # expectation over "blowing" states s'
                q[s][a] += Ra 
                
            # Greedy policy Improvement
            policy[s] = np.eye(env.nA)[np.argmax(q[s][:])]# acting greedy is deterministic        
            
        # Stop Policy Iteration when Bellman opimality equation is satisfies <-- Vpi = V*    
        if all([Vpi[s] == np.amax(q[s][:]) for s in range(env.nS)]):
            return policy, Vpi

In [5]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [6]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

# Run Simulation with computed Policy 

In [7]:
no_episodes = 3

for i_episode in range(no_episodes):
    observation = env.reset() # Start episode    
    
    for t in range(100):
        env.render() # simulate a time step
        print("")
        action = np.argmax(policy[obyservation]) # sample action from policy distribution
        observation, _, done, _ = env.step(action) # execute sampled action 
        if done:
            env.render() # simulate a time step
            print("\nEpisode finished after {} timesteps\n".format(t+1))
            break
    
env.render(close=True)# Close window

T  o  o  o
o  o  o  o
o  o  o  o
o  o  x  T



NameError: name 'obyservation' is not defined